<a href="https://colab.research.google.com/github/ckjen168/LLMColab/blob/main/stk_ch08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-08 建構投資組合：讓 AI 輔助選股

## 8-1 建構投資組合

### 1️⃣ 匯入套件 & 掛載雲端硬碟



In [ ]:
!pip install gdown
import gdown
import os
import datetime as dt
import time
from IPython.display import display
import pandas as pd
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')

Mounted at /content/drive


### 2️⃣ 下載資料庫
若資料庫未下載成功或出現 bug，請手動點擊連結下載，並放入 **我的雲端硬碟>StockGPT** 資料夾中

下載連結：https://drive.usercontent.google.com/download?id=1S5JE9ZF2hohRpvO8FikgLhQmN2DJrVgW

In [ ]:
# 指定下載路徑
!mkdir -p "/content/drive/MyDrive/StockGPT/"
output_path = '/content/drive/MyDrive/StockGPT/'

# 檢查資料庫是否存在
stock_db_path = output_path + 'stock.db'
if not os.path.exists(stock_db_path):
    print("下載資料庫中...")
    id = '1S5JE9ZF2hohRpvO8FikgLhQmN2DJrVgW'

    gdown.download(id=id, output=stock_db_path)
    print("下載完成")
else:
    print("無需下載")

無需下載


### 3️⃣ 安裝旗標套件

In [ ]:
!git clone https://github.com/FlagTech/F3933.git
%cd F3933
!pip install -r requirements.txt
from Ch06 import StockAnalysis, StockInfo
from Ch07 import PdfLoader
from Stock_DB import StockDB
%cd ..
!pip install numpy --upgrade

Cloning into 'F3933'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 622 (delta 73), reused 31 (delta 31), pack-reused 521 (from 2)
Receiving objects: 100% (622/622), 190.62 KiB | 5.78 MiB/s, done.
Resolving deltas: 100% (377/377), done.
/content/F3933
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━

### 4️⃣ 更新資料庫
資料庫更新需耗費較長時間, 若無法更新，請刪除我的雲端硬碟>StockGPT>stock.db 檔案，並至第二個儲存格的連結手動下載

In [ ]:
stock_db = StockDB()
stock_db.renew()
stock_db.close()

###5️⃣ 檢視資料表格式

In [ ]:
# 從資料庫中取得表格
stock_db = StockDB()
df_company = stock_db.get("公司")
df_daily = stock_db.get("日頻",psdate=True)
df_quarterly = stock_db.get("季頻",psdate=True)

# 去除空值
df_company = df_company.dropna()
df_daily = df_daily.dropna()
df_quarterly = df_quarterly.dropna()

# 檢視資料
display(df_company.head())
display(df_daily.head())
display(df_quarterly.head())

,股號,股名,產業別,股本,市值
0,1101,台泥,水泥工業,7.136180e+09,2.501231e+11
1,1102,亞泥,水泥工業,3.546560e+09,1.425717e+11
2,1103,嘉泥,水泥工業,6.588940e+08,1.242015e+10
3,1104,環泥,水泥工業,6.732170e+08,1.908570e+10
4,1108,幸福,水泥工業,4.047380e+08,6.212728e+09


,股號,日期,開盤價,最高價,最低價,收盤價,還原價,成交量,殖利率,日本益比,股價淨值比,三大法人買賣超股數,融資買入,融卷賣出
0,1101,2015-01-05,31.923370,31.923370,31.482035,31.629147,20.498331,3847400.0,5.35,14.88,1.42,-1005483,211.0,20.0
1,1101,2015-01-06,31.261366,31.334923,30.967140,31.077475,20.140799,10386622.0,5.44,14.62,1.4,-2524874,208.0,32.0
2,1101,2015-01-07,31.077475,31.298143,30.783251,30.967140,20.069292,11841293.0,5.46,14.57,1.39,-5742000,264.0,13.0
3,1101,2015-01-08,31.077475,31.371700,30.967140,31.077475,20.140799,11815462.0,5.44,14.62,1.4,-5120668,245.0,38.0
4,1101,2015-01-09,31.114252,31.334923,30.746471,30.746471,19.926281,14240820.0,5.50,14.46,1.38,-7993555,566.0,56.0


,股號,日期,營業收入,營業費用,稅後淨利,每股盈餘
0,1101,2024-09-01,41075042.0,3699007.0,4619116.0,0.42
1,1101,2024-03-01,25544599.0,2687318.0,2239346.0,0.26
2,1101,2023-12-01,28347871.0,2871134.0,2499914.0,0.25
3,1101,2023-09-01,27002293.0,2712611.0,2292372.0,0.17
4,1101,2023-06-01,27668242.0,2616710.0,4206683.0,0.45


###6️⃣ 輸入 OpenAI API KEY & 建立物件

In [ ]:
import getpass
openai_api_key = getpass.getpass("請輸入金鑰：")
six = StockAnalysis(openai_api_key)
seven = PdfLoader(openai_api_key)

請輸入金鑰：··········


### 7️⃣ AI 自動化選股機器人

<輸入範例>
```
請選出近一週漲幅最高的10檔股票
請選出大市值股(前10%)且近期營收成長最高的10檔股票
請選出半導體業且近期每股盈餘成長率最高的10檔股票
請選出近一年股價淨值比最低的10檔股票
請從近一個月成交量前10%的股票中找出日本益比最低的5檔不同的股票
```


In [ ]:
# 輸入使用者需求
user_msg = input("請輸入選股需求:")

# 自動處理 3 個資料表並生成程式碼
history, code_str = six.ai_helper(user_msg)

# AI 自動選股 & 除錯
success = False
for _ in range(3):
    try:
        # 新建 df 表格
        df1 = df_company
        df2 = df_daily
        df3 = df_quarterly

        print(code_str)
        exec(code_str)
        new_df = calculate(df1, df2, df3)
        success = True
        break
    except Exception as e:
        print(f"AI 除錯中...")
        code_str = six.ai_debug(history, code_str, str(e))
        print(code_str)
        print("-------------------------")

if not success:
    print("請更換或重新輸入選股需求")
else:
    display(new_df)

請輸入選股需求:請從近一個月成交量前10%的股票中找出日本益比最低的5檔不同的股票


def calculate(table_company, table_daily, table_quarterly):
    table_daily['日本益比'] = pd.to_numeric(table_daily['日本益比'], errors='coerce')
    table_daily['成交量'] = pd.to_numeric(table_daily['成交量'], errors='coerce')
    
    recent_month_data = table_daily[table_daily['日期'] >= pd.Timestamp.today() - pd.DateOffset(months=1)]
    top_10_percent_volume = recent_month_data.nlargest(int(len(recent_month_data) * 0.1), '成交量')
    
    top_10_percent_volume_unique = top_10_percent_volume.groupby('股號').first().reset_index()
    top_5_low_pe_stocks = top_10_percent_volume_unique.nsmallest(5, '日本益比')
    
    return top_5_low_pe_stocks



,股號,日期,開盤價,最高價,最低價,收盤價,還原價,成交量,殖利率,日本益比,股價淨值比,三大法人買賣超股數,融資買入,融卷賣出
68,2603,2025-03-14,226.000000,229.000000,219.500000,220.000000,220,65694548.0,14.77,3.42,0.82,"-22,924,929","4,980",212.0
88,2915,2025-03-13,65.000000,65.500000,63.099998,63.900002,63.900002,12416026.0,3.91,5.03,0.67,"-3,228,206",616.0,18.0
70,2609,2025-03-13,75.400002,76.000000,74.199997,74.400002,74.400002,59944109.0,2.69,5.16,0.8,"-2,832,596","2,752",97.0
67,2520,2025-03-10,58.000000,58.099998,56.299999,56.799999,56.799999,8453656.0,3.17,5.54,1.25,"280,802",424.0,5.0
101,3056,2025-03-12,37.299999,37.349998,35.200001,36.500000,36.5,18022965.0,15.07,5.93,1.45,"-3,168,745","1,173",16.0


## 8-2 AI 趨勢報告推薦系統

### 8️⃣ 取得個股分析報告

In [ ]:
reply = six.stock_gpt(stock_id="2330")
print(reply)

[*********************100%***********************]  1 of 1 completed


根據最近的價格資訊，台積電（2330-TW）的股價在近期呈現波動。從2025年3月11日至3月26日，台積電的收盤價從971.0元上升至983.0元，整體上呈現輕微上升的趨勢。然而，每日報酬率的變動顯示了股價波動的情況，投資人應謹慎應對。

在基本面方面，台積電在最近的每季營收資訊中顯示了穩健的表現。營收成長率呈現穩定增長，並且EPS在過去幾個季度也呈現增長的趨勢，顯示公司的盈利能力良好。然而，需要注意的是，EPS季增率在最近一季出現了小幅下降，投資人應密切關注未來的盈利表現。

從最近的新聞資訊來看，外資對台積電的買超情況較為積極，顯示外資對公司長期發展前景持樂觀態度。此外，俄烏黑海停火協議對市場情緒產生了一定影響，使得部分概念股走強，包括鋼鐵和水泥類股等。

總體而言，台積電在近期呈現輕微上升的股價趨勢，基本面表現穩健，並且受到外資青睞。然而，投資人應密切關注市場的變化，並謹慎評估風險。建議投資人在做出投資決策前，應詳細分析公司的財務狀況、行業環境以及市場趨勢，以確保投資安全和穩健增長。


### 9️⃣ 收集多檔股票的趨勢報告
10 檔股票約會跑 5 分鐘, 若跑太久為 API 當掉

In [ ]:
# 建立股票清單
stock_list = ['2364', '2535', '3041', '5215', '2363',
              '1568', '2369', '2816', '9955', '2233']

# 設定儲存路徑
today_time = dt.date.today().strftime("%Y%m%d")
path = '/content/drive/MyDrive/StockGPT/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

for stock in stock_list:
  file_path = f"{path}trend_{stock}_{today_time}.txt"

  if os.path.exists(file_path):
    print(f"{stock} 檔案已存在")
  else:
    with open(file_path, "w", encoding="utf-8") as f:
      f.write(six.stock_gpt(stock_id=stock))

  with open(file_path, "r", encoding="utf-8") as f:
      content[stock] = f.read()


2364 檔案已存在
2535 檔案已存在
3041 檔案已存在
5215 檔案已存在
2363 檔案已存在
1568 檔案已存在
2369 檔案已存在
2816 檔案已存在
9955 檔案已存在
2233 檔案已存在


### 🔟 推薦出一檔股票

In [ ]:
def stock_choice(data):
    # 設定提示模板
    msg = [{
        "role": "system",
        "content":
        "你現在是一位專業的證券分析師, 你會針對各股的專業趨勢分析報告, \n\
         選擇出最適合投資的一檔股票,並說明選擇它的理由。"
    }, {
        "role": "user",
        "content": str(data)
    }]

    reply_data = six.get_reply(msg)
    return reply_data

print(stock_choice(content))

根據提供的資料，我將選擇投資於矽統（股票代號：2363）公司。以下是我對矽統公司近期趨勢的分析報告：

1. **股價走勢分析：**
   - 從近期收盤價資訊來看，矽統的股價在2025年3月11日至2025年3月26日期間呈現下跌趨勢，整體下跌幅度約5%。
   - 每日報酬率顯示，矽統股價波動較大，但整體趨勢相對平穩。

2. **基本面分析：**
   - 矽統在過去幾個季度的營收成長率呈現波動，但整體呈現穩定增長趨勢。
   - EPS在過去幾個季度中有所波動，但整體呈現正增長，顯示公司盈利能力有所提升。

3. **新聞資訊分析：**
   - 近期新聞資訊顯示，矽統的2月營收年增率高達1337.03%，顯示公司業績表現優異。
   - 公司在大型公司2月營收一覽中表現優異，市場對其未來發展持樂觀態度。

**總結與建議：**
綜合以上分析，雖然矽統近期股價呈現下跌趨勢，但基本面表現穩健，且營收表現優異，顯示公司具有成長潛力。投資者應密切關注公司未來的營收表現及市場動向，以制定適合的投資策略。此報告僅供參考，投資需謹慎評估風險。

因此，我建議投資者關注矽統公司，密切關注其未來的營收表現和市場動向，以做出明智的投資決策。希望這份分析報告對您有所幫助。


### 1️⃣1️⃣ 推薦股票的評分排序

In [ ]:
def stock_sort(data):
    # 設定提示模板
    msg = [{
        "role": "system",
        "content":
        "你現在是一位專業的股票分析師, 會根據各股的專業趨勢分析報告去評斷\
         適不適合投資, 並給予0-100之間的評分。\n\
         以 50 分為基準, 有任何正面消息可以加分如:\n\
         股價整體上升、法人買超、營收成長上升、新聞有正面消息；\n\
         若有任何負面消息必須扣分如:\n\
         股價整體下降、法人賣超、營收成長下降、新聞有負面消息。\n\
         最後請將所有股票依照評分排序出來。"
    }, {
        "role": "user",
        "content": str(data)
    }]
    reply_data = six.get_reply(msg)
    return reply_data

print(stock_sort(content))

根據提供的資料，我將對各股票的趨勢進行分析並給予評分。

1. 2364 (倫飛): 股價波動較大，營收成長率為6%，但EPS季增率波動較大，且受市場環境和產業消息影響。評分: 45

2. 2535 (未提供資料): 無法提供評分。

3. 3041 (未提供資料): 無法提供評分。

4. 5215 (科嘉-KY): 股價波動較大，營收成長率在不同季度呈現波動，且市場上被認為是值得持續關注的強勢股之一。評分: 55

5. 2363 (矽統): 股價呈現明顯的下跌趨勢，營收成長率有一定程度的增長，但EPS季增率波動較大，且受外資賣超情況影響。評分: 40

6. 1568 (倉佑): 股價呈現下跌趨勢，營收成長率輕微增長，但EPS季增率呈現波動，且受多方面因素影響。評分: 42

7. 2369 (菱生): 股價呈現下跌趨勢，營收成長率有所增長，但EPS呈現下滑趨勢，且受市場情緒影響。評分: 45

8. 2816 (旺旺保): 股價波動較大，營收成長率輕微增長，但EPS季增率呈現下降，且受多項因素影響。評分: 48

9. 9955 (佳龍公司): 股價呈現下跌趨勢，營收成長不穩定，且EPS波動較大，受多項因素影響。評分: 40

10. 2233 (宇隆): 股價波動較大，營收有一定程度的增長，且受營運週轉資金需求和產業投資價值影響。評分: 50

根據以上分析，我將這些股票依照評分排序如下：
1. 2816 (旺旺保) - 48
2. 2364 (倫飛) - 45
3. 2369 (菱生) - 45
4. 2233 (宇隆) - 50
5. 5215 (科嘉-KY) - 55
6. 1568 (倉佑) - 42
7. 2363 (矽統) - 40
8. 9955 (佳龍公司) - 40


## 8-3 AI 年報分析推薦系統

### 1️⃣2️⃣ 取得多檔股票的年報檔案

In [ ]:
# 建立股票清單
stock_list = ['2364', '2535', '3041', '5215', '2363',
              '1568', '2369', '2816', '9955', '2233']

# 取得並儲存年報資料
for stock in stock_list:
  if not os.path.exists(f'/content/drive/MyDrive/StockGPT/PDF/113_{stock}.pdf'):
    try:
        seven.annual_report(stock,'113')
    except:
        time.sleep(10)
        while True:
            try:
                seven.annual_report(stock,'113')
                break
            except:
                time.sleep(10*2)

### 1️⃣3️⃣ 建立向量資料庫
10 檔股票約需 20 分鐘

In [ ]:
db_list=[]
for i in stock_list:
    if not os.path.exists(
        f'/content/drive/MyDrive/StockGPT/DB/113_{i}'):
        print('start',i)
        db_list.append(
            seven.pdf_loader(
                f'/content/drive/MyDrive/StockGPT/PDF/113_{i}.pdf',
                600, 60))

### 1️⃣4️⃣ 讀取向量資料庫

In [ ]:
!pip install faiss-cpu
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db_list=[]
for i in stock_list:
    db_list.append(
        FAISS.load_local(
          folder_path=f'/content/drive/MyDrive/StockGPT/DB/113_{i}',
          embeddings=OpenAIEmbeddings(),
          allow_dangerous_deserialization=True
          ))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.7 MB/s eta 0:00:00


### 1️⃣5️⃣ 建立問題串列關鍵字

In [ ]:
key_word = ['營收成長如何？',
            '營運策略和市場定位是什麼？',
            '面對哪些主要風險和挑戰？',
            '公司在研發和創新方面有哪些成就和計劃？',
            '公司治理結構和管理層的組成是？']

### 1️⃣6️⃣ 取得相關資料 & 建立分析機器人

In [ ]:
# 建立從向量資料庫取得相關資訊函式
def generate_data(db, key_word):
    results = []
    for word in key_word:
        results.append(seven.analyze_chain(db, word))
    return results

# 建立年報分析機器人
def stock_report_summary(data):

    msg = [{
        "role": "system",
        "content": "你現在是一位專業的年報分析師,\n\
                    你會針對年報報告彙整出一份專業的分析報告。\n\
                    請以詳細、嚴謹及專業的角度撰寫此報告,並提及重要的數字\
                    ,reply in 繁體中文"
    }, {
        "role": "user",
        "content": str(data)
    }]

    reply_data = six.get_reply(msg)

    return reply_data

### 1️⃣7️⃣ 多檔股票的年報分析報告

In [ ]:
# 年報分析檔案的儲存路徑
path = "/content/drive/MyDrive/StockGPT/AnnualReport/"
os.makedirs(path, exist_ok=True)


# 建立多檔股票的年報分析報告並儲存
content = {}

for stock in tqdm(stock_list, total=len(stock_list)):
  file_path = f"{path}annual_{stock}_113.txt"

  if os.path.exists(file_path):
    print(f"{stock} 檔案已存在")
  else:

      db = db_list[stock_list.index(stock)]
      report = stock_report_summary(generate_data(db, key_word))
      print(report)

      with open(file_path, "w", encoding="utf-8") as f:
        f.write(report)

  # 取得股名
  name = six.stock_info.get_stock_name(stock, six.name_df)

  with open(file_path, "r", encoding="utf-8") as f:
      content[f"{name}({stock})"] = f.read()

 10%|█         | 1/10 [00:31<04:47, 31.89s/it]

根據提供的年報資訊，我們對倫飛電腦實業股份有限公司的財務狀況進行分析如下：

1. 資產總額增加：從111年度的1,156,327千元增加至112年度的1,306,981千元，增加了150,654千元，增幅為13.03%。這顯示公司在過去一年中積極擴大資產規模，可能是為了支持業務擴張或投資計劃。

2. 負債總額增加：從111年度的830,585千元增加至112年度的878,750千元，增加了48,165千元，增幅為5.80%。負債增加的幅度較資產增加緩慢，這可能會對公司的財務風險產生一定影響，需要密切關注。

3. 權益總額增加：從111年度的325,742千元增加至112年度的428,231千元，增加了102,489千元，增幅為31.46%。權益增加顯示公司在過去一年中取得了良好的獲利表現，有助於提高公司的資本結構穩定性。

重大變動項目說明：
- 非流動負債增加主要是因為租賃負債增加所致。這可能是公司擴大業務所需的資金支出所致。
- 股本增加是因為盈餘轉增資發行新股所致。公司可能透過增加股本來籌集資金以支持業務發展。
- 保留盈餘增加是因為本期獲利所致。公司在112年度取得了良好的獲利表現，有助於增加保留盈餘，提高公司的資本穩定性。

此外，公司在財務報告中提到了信用風險的管理措施，包括對應收帳款和投資進行評估，建立授信政策等。這顯示公司重視風險管理，有助於降低潛在的財務風險。

總體而言，倫飛電腦實業股份有限公司在112年度取得了良好的財務績效，資產和權益均有顯著增加，但仍需密切關注負債增加對公司財務風險的影響。公司在風險管理方面已經有相應措施，應繼續保持財務穩健，以應對未來可能的挑戰。


 20%|██        | 2/10 [01:01<04:02, 30.35s/it]

根據提供的年報資訊，我們可以對每份摘要進行專業分析如下：

1. 第一份摘要涉及公司的財務狀況比較分析。根據報告顯示，公司在一一二年度的流動資產增加了941,764仟元，增幅為4.43%，資產總額也增加了1,222,221仟元，增幅為5.38%。然而，流動負債增加了2,554,787仟元，增幅高達29.45%，導致負債總額增加了1,408,809仟元，增幅為10.60%。這顯示公司的資產增長速度較快，但負債增長更快，導致負債占比增加。建議公司應注意控制負債增長速度，以確保財務穩健。

2. 第二份摘要涉及市場及產銷概況。公司主要服務地區為台灣，市場競爭激烈，並有未來供需狀況與成長性的挑戰。公司營業計畫包括研發低碳工法、推動智慧工地管理系統等。未來發展策略受氣候變遷法規、缺工及成本上漲等因素影響。建議公司應密切關注市場變化，提升競爭力。

3. 第三份摘要提及公司的資本支出、轉投資政策、風險管理等方面。公司未從事高風險投資，並有獨立董事監督公司營運。建議公司應持續加強風險管理，確保投資計畫的有效執行。

4. 第四份摘要涉及公司的經營方針、產銷政策、未來發展策略等。公司積極應對外部競爭環境，拓展海外業務，並提升營建價值及競爭力。重點數據包括全據點溫室氣體盤查、海外業務拓展等。建議公司應持續強化研發能力，提升產品價值，並加強與上下游產業合作。

總體而言，以上公司在財務、市場、治理等方面均有優勢和挑戰。建議公司應根據報告中提到的重要數據和分析結果，制定相應的策略和措施，以確保公司的永續發展和競爭優勢。


 30%|███       | 3/10 [01:34<03:43, 31.91s/it]

...

根據提供的年報資訊，我們可以看到公司在風險管理、資本管理、企業價值觀和永續發展等方面都有明確的政策和措施，致力於維持穩健營運和創造價值。以下是對每個重點摘要的專業分析：

1. **風險管理**：
   - 公司的風險管理機制包括市場風險、信用風險、流動性風險、作業風險、保險風險等多個方面，顯示公司對多元風險的關注和管理。
   - 公司強調公平待客原則，提升金融消費者保護意識，這有助於降低違法成本和風險，增進金融消費者對公司的信心。
   - 公司持續關注科技改變和產業變化對財務業務的影響，並採取因應措施以確保長期穩定發展。
   - 公司設立危機應變措施，以應對可能影響公司形象的危機事件，這有助於提高公司的危機應變能力。

2. **資本管理**：
   - 公司的資本管理政策旨在維持健全的資本基礎，支持未來營運發展，這有助於確保公司有足夠的資金支持業務運作。
   - 控管資本報酬率和普通股股利水準，有助於提高公司的財務效率和股東價值。

3. **企業價值觀**：
   - 公司強調法令遵循、保險專業、公司治理，並追求長期穩定獲利以創造股東價值，這有助於建立公司的良好聲譽和形象。
   - 企業社會責任包括穩健、安全、創新的信念，這有助於提升公司在社會中的形象和價值觀。

4. **永續發展**：
   - 公司在永續發展方面積極推動作業流程簡化、數位化經營、多元化商品等策略，這有助於提升公司的競爭力和市場佔有率。
   - 參與公益活動、推廣保險知識、綠色關懷等活動，有助於提升公司的社會形象和責任感。

總的來說，公司在風險管理、資本管理、企業價值觀和永續發展方面都展現出積極的執行情形，這些措施有助於提高公司的競爭力、營運效率和社會價值，展現了公司對於長期穩定發展的承諾和努力。希望公司能持續保持這些良好的管理實踐，並不斷追求卓越。


 90%|█████████ | 9/10 [04:16<00:29, 29.71s/it]

根據提供的年報資訊，我們可以看到公司在財務風險管理、公司治理、長期計畫、多角化經營和組織系統等方面都有相當詳細的規劃和措施。以下是對這些資訊的專業分析報告：

1. 財務風險管理：
公司在財務風險管理方面需確實遵循相關規定，並針對市場風險，特別是匯率風險和利率風險進行評估和因應措施。透過穩健保守的資金規劃和外幣資產負債互抵效果等措施，公司能夠有效降低風險對營運的影響。

2. 公司治理：
公司在公司治理方面已經有一些改善措施，例如董事會成員至少包含一位女性董事、審計委員會審核財務報告等。然而，公司也提出了優先加強的事項，如訂定董事會成員多元化政策、揭露獨立董事與內部核主管溝通情形等，這些措施有助於提升公司治理水平。

3. 長期計畫：
公司的長期計畫包括行銷策略、生產政策、產品發展方向和經營管理策略等多個方面。特別是公司致力於發展綠能應用事業、降低生產成本、提高回收技術，並透過策略性投資推動業務與科技雙引擎，以實現長期穩定的市值成長。

4. 多角化經營：
公司將聚焦於綠能環保及半導體產業之投資機會，並根據市場狀況和營業規模的需要調整投資計劃。公司在風險事項分析評估中，針對利率、匯率變動和通貨膨脹等因素進行了評估和因應措施，以降低風險影響。

5. 組織系統：
公司的組織系統包括董事會、審計委員會、核室、酬委員會等部門，並在當年度加強董事會職能，提升資訊透明度等目標。公司在銷售增長、營收變化和開發項目等方面取得了一定的成就，顯示公司在組織架構和營運方面有著明確的規劃和執行。

總體而言，公司在財務風險管理、公司治理、長期計畫、多角化經營和組織系統等方面都有著積極的措施和計畫，有助於提升公司的競爭力和長期價值。然而，公司仍應持續關注市場變化和風險因素，並不斷優化管理策略以確保持續的成長和價值創造。


100%|██████████| 10/10 [04:47<00:00, 28.72s/it]

根據提供的年報資訊，我們可以看到公司在資本管理、營運管理、財務管理和營運計畫方面都有明確的策略和措施，以確保企業的穩健運營和未來發展。以下是針對每個重點摘要提供更具體的分析：

1. 資本管理：
   - 公司主要目標是確保維持健全的信用評等和良好的資本比率，以支持企業營運和最大化股東權益。透過調整股利支付或發行新股來管理和調整資本結構，這顯示公司在資本管理上有靈活性和穩健性。
   
2. 營運管理：
   - 公司建立扁平化組織、完善目標管理制度和員工績效考核制度，並定期舉辦內外部員工教育訓練，強化營運總部管理功能。這些措施有助於提升營運效率和員工素質，推動公司向全球化競爭和員工終身學習觀念。
   
3. 財務管理：
   - 公司除了利用稅後盈餘補充營運資金外，還搭配金融機構貸款和現金增資，以滿足未來發展所需資金。同時，妥善安排公司財務規劃，降低營運風險。這顯示公司在財務管理上有著穩健的策略和風險控制措施。

4. 營運計畫：
   - 公司在營運計畫中提到將持續擴充產能、提升生產效率，並透過自動化生產與檢驗、AI、大數據等技術來提高製程與品質能力。公司積極開發新產品、拓展新市場，並強調對員工福祉和環境資源的關懷，推動ESG永續發展策略。這些計畫顯示公司對於未來發展有清晰的方向和策略。

總體而言，公司在資本管理、營運管理、財務管理和營運計畫方面都展現出積極的態度和明確的策略，有助於確保企業的穩健運營和未來發展。然而，公司仍需密切關注市場變化和競爭環境，持續優化管理策略，以因應未來挑戰並實現長期可持續發展。


### 1️⃣8️⃣ 根據分析結果推薦出一檔股票

In [ ]:
print(stock_choice(content))

根據所提供的年報分析報告，我會選擇倫飛(2364)作為最適合投資的一檔股票。以下是我選擇它的理由：

1. 財務比率表現良好：根據年報的分析，倫飛公司在財務比率方面表現良好，包括利息保障倍數、資產報酬率、權益報酬率、純益率等指標均呈現上升趨勢。這顯示公司有良好的獲利能力和資產運用效率。

2. 公司努力和措施：倫飛公司積極強化董事會職能、加強公司治理，並重視產品創新和研發，以提高競爭力和市場佔有率。這些努力有助於提高公司的透明度、獲利能力和競爭力。

3. 現金流量允許比率上升：根據年報的分析，倫飛公司的現金流量允許比率上升，顯示公司有足夠的現金流量來應對短期債務，這有助於確保公司的財務穩健性。

綜合以上分析，倫飛公司在財務表現、公司治理和市場競爭力方面均表現良好，並且有積極的發展策略和措施。因此，我認為倫飛(2364)是一檔值得投資的股票。


### 1️⃣9️⃣ 年報分析報告評分排序

In [ ]:
def stock_report_sort(data):
    # 設定提示模板
    msg = [{
        "role": "system",
        "content":
        "你現在是一位專業的股票分析師, 會根據各股的年報分析報告去評斷\
         適不適合投資, 並給予0-100之間的評分。\n\
         以 50 分為基準, 有任何正面消息可以加分如:\n\
         財務呈現穩定增長的趨勢, 具有競爭優勢；\n\
         若有任何負面消息必須扣分如:\n\
         財務呈現下降的趨勢, 不具有競爭優勢。\n\
         最後請將10檔股票依照評分排序出來。"
    }, {
        "role": "user",
        "content": str(data)
    }]
    reply_data = six.get_reply(msg)
    return reply_data

print(stock_report_sort(content))